In [175]:
from collections import defaultdict
import itertools

In [619]:
def count_score (dictionary):
    pairwise_sent_agreement = []
    for k, v in dictionary.items():     
        spans = v.items()

        combinations = list(itertools.combinations(spans, 2))
        scores = []
        for combination in combinations:
            score = 0
            
            a = set(combination[0][1].keys())
            b = set(combination[1][1].keys())
            i = a.intersection(b)
            annotator_pair = (combination[0][0], combination[1][0])
            
            for span in i:
                correction1 = combination[0][1][span]
                correction2 = combination[1][1][span]
                if correction1 == correction2:
                    score += 1 
            scores.append((annotator_pair, score / len(a.union(b))))
        pairwise_sent_agreement.append(scores)
    
    average_pairwise_agreement = defaultdict(list)
    average_sent_agreements = []
    
    for sentence in pairwise_sent_agreement:
        sent_agreements = []
        for pair in sentence:
            average_pairwise_agreement[pair[0]].append(pair[1])
            sent_agreements.append(pair[1])
            
        average_sent_agreements.append(sum(sent_agreements) / len(sent_agreements))
        
    total_agreement = sum(average_sent_agreements) / len(average_sent_agreements)
    
    for k,v in average_pairwise_agreement.items():
        average_pairwise_agreement[k] = sum(v) / len(v)
            
    return average_pairwise_agreement, total_agreement

In [620]:
dictionary = defaultdict(dict)
with open('data.txt') as f:
    f = f.read()
    for pair in f.split('\n\n'):
        p = pair.split('\n')
        sentence = p[0][2:]
        answers = p[1:]
        dictionary[sentence] = defaultdict(dict)
        
        for answer in answers:
            parts = answer[2:].split('|||')
            annonator_number = int(parts[-1])
            span = tuple(parts[0].split())
            word = parts[2]
            
            if word  != '-NONE-': 
                dictionary[sentence][annonator_number][span] = word
                
        if len(dictionary[sentence].keys()) < 2:
            del  dictionary[sentence]
        
print(f'Pairwise agreement: {count_score(dictionary)[0]} \nTotal corporus agreement: {count_score(dictionary)[1]}')  


Pairwise agreement: defaultdict(<class 'list'>, {(0, 1): 0.23259100781842124, (0, 2): 0.38531274168926005, (1, 2): 0.3980529397966955, (0, 3): 0.40152721799273516, (0, 4): 0.512962962962963, (1, 3): 0.4191578118850847, (1, 4): 0.3962301587301587, (2, 3): 0.45585895491891426, (2, 4): 0.33730158730158727, (3, 4): 0.45777777777777773}) 
Total corporus agreement: 0.27275232505373465




Рівень згоди анотувальників досить низький. Вочевидь, це через те, що я не врахувала часткову згоду при перетині виправлень. 